<a href="https://colab.research.google.com/github/IlhamDev007/Github_demo1/blob/main/ADPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [5]:
# Import Require Libraies
import pandas as pd
from abc import ABC, abstractmethod
import dash
from dash import html
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

<ipython-input-5-4c5561952e63>:6: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
<ipython-input-5-4c5561952e63>:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-5-4c5561952e63>:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [6]:
# Interface for data ingestion
class DataIngestionStrategy(ABC):
    @abstractmethod
    def ingest_data(self, file_path):
        pass

# Concrete class for CSV data ingestion
class CSVDataIngestion(DataIngestionStrategy):
    def ingest_data(self, file_path):
        return pd.read_csv(file_path)


# Context class for data ingestion
class DataIngestionContext:
    def __init__(self, strategy: DataIngestionStrategy):
        self._strategy = strategy

    def set_strategy(self, strategy: DataIngestionStrategy):
        self._strategy = strategy

    def ingest(self, file_path):
        return self._strategy.ingest_data(file_path)


class DataProcess(ABC):
    @abstractmethod
    def process_data(self, data):
        pass

In [7]:
class SalesTrendsOverTime(DataProcess):
    def process_data(self, data):
        # Ensure 'Date' is in datetime format
        data['Date'] = pd.to_datetime(data['Date'])

        # Group by year and month, then sum the sales
        grouped = data.groupby([data['Date'].dt.year.rename('Year'), data['Date'].dt.month.rename('Month')])[
            'Sales'].sum().reset_index(name='TotalSales')

        # Reconstructing the 'Date' column for ease of plotting
        grouped['Date'] = pd.to_datetime(grouped.assign(DAY=1)[['Year', 'Month', 'DAY']])

        # Ensure 'Date' is the first column if needed
        cols = ['Date'] + [col for col in grouped.columns if col != 'Date']
        return grouped[cols]